# Metrics

https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226

https://towardsdatascience.com/20-popular-machine-learning-metrics-part-1-classification-regression-evaluation-metrics-1ca3e282a2ce

In [26]:
import numpy as np
import json
import copy
from json import JSONEncoder
import os

In [2]:
class EvaluatedModel:
    y_pred = []
    y_true = []
    model_name = ''
    avg_time = 0.0
    gzip_size = 0
    
    accuracy = None
    precision = None
    recall = None
    f1_score = None
    

    def __init__(self, dict1=None, y_pred=[], y_true=[], model_name='', avg_time=0.0, gzip_size=0):
        if (dict1 == None):
            self.y_pred = copy.deepcopy(y_pred)
            self.y_true = copy.deepcopy(y_true)
            self.model_name = model_name
            self.avg_time = avg_time
            self.gzip_size = gzip_size
        else:
            self.__dict__.update(dict1)


## 1. Compute metrics for models

In [30]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score 
from sklearn.metrics import f1_score

def annote_models(path):
    with open(path) as f:
        evaluated_models = json.load(f, object_hook=EvaluatedModel)

    for evaluated_model in evaluated_models:
        evaluated_model.accuracy = accuracy_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true)
        evaluated_model.precision = precision_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='micro')
        evaluated_model.recall = recall_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='micro')
        evaluated_model.f1_score = f1_score(y_pred=evaluated_model.y_pred, y_true=evaluated_model.y_true, average='micro')
        # change model name for table
        evaluated_model.model_name = evaluated_model.model_name.replace("Efficent", "Efficient")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus32", "WQ32")
        evaluated_model.model_name = evaluated_model.model_name.replace("KMeansPlusPlus128", "WQ128")
        evaluated_model.model_name = evaluated_model.model_name.replace("full_integer_quantization", "FIQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("dynamic_rage_quantization", "DRQ")
        evaluated_model.model_name = evaluated_model.model_name.replace("float16_quantization", "F16Q")
        evaluated_model.model_name = evaluated_model.model_name.replace("_integer_io", "-Iio")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay90", "PD90")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay75", "PD75")
        evaluated_model.model_name = evaluated_model.model_name.replace("PolynomialDecay50", "PD50")
        evaluated_model.model_name = evaluated_model.model_name.replace("flowers_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("beans_model_", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity90", "CS90")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity75", "CS75")
        evaluated_model.model_name = evaluated_model.model_name.replace("ConstantSparsity50", "CS50")
        evaluated_model.model_name = evaluated_model.model_name.replace(".tflite", "")
        evaluated_model.model_name = evaluated_model.model_name.replace("_", " -> ")
        
    return evaluated_models

def print_table(models):
    print("|Model name|Accuarcy|Precision|Recall|F1 score|Compressed size|Average time|")
    print("|-----|-----|-----|-----|-----|-----|-----|")
    for model in models:
        print("|"+model.model_name+"|"+str(round(model.accuracy * 100, 2))+"|"+str(round(model.precision * 100, 2))
              +"|"+str(round(model.recall * 100, 2))+"|"+str(round(model.f1_score * 100, 2))+"|"
              +str(round(model.gzip_size / 1024))+" kB|"+str(round(model.avg_time, 2))+" ms|")


In [31]:
flowers_models = annote_models('results/evaluated_flowers_models_on_MobileNetV2.json')
print_table(flowers_models)


|Model name|Accuarcy|Precision|Recall|F1 score|Compressed size|Average time|
|-----|-----|-----|-----|-----|-----|-----|
|MobileNetV2 -> CS50 -> FIQ-Iio|0.36|0.36|0.36|0.36|2334 kB|51.8 ms|
|MobileNetV2 -> CS75 -> WQ32 -> FIQ-Iio|0.36|0.36|0.36|0.36|1855 kB|50.74 ms|
|MobileNetV2 -> PD50 -> WQ128 -> DRQ|0.24|0.24|0.24|0.24|2224 kB|75.25 ms|
|MobileNetV2 -> CS75 -> FIQ-Iio|0.36|0.36|0.36|0.36|2155 kB|46.59 ms|
|MobileNetV2 -> CS90 -> DRQ|0.33|0.33|0.33|0.33|1971 kB|76.82 ms|
|MobileNetV2 -> PD50 -> FIQ-Iio|0.36|0.36|0.36|0.36|2334 kB|47.71 ms|
|MobileNetV2 -> PD75 -> FIQ|0.37|0.37|0.37|0.37|2154 kB|46.01 ms|
|MobileNetV2 -> WQ128 -> FIQ-Iio|0.33|0.33|0.33|0.33|2370 kB|46.65 ms|
|MobileNetV2 -> CS50 -> WQ32 -> FIQ|0.28|0.28|0.28|0.28|1955 kB|45.95 ms|
|MobileNetV2 -> CS90 -> WQ128 -> F16Q|0.37|0.37|0.37|0.37|3734 kB|64.18 ms|
|MobileNetV2 -> CS75|0.36|0.36|0.36|0.36|8559 kB|65.56 ms|
|MobileNetV2 -> PD90 -> WQ128 -> FIQ|0.54|0.54|0.54|0.54|2022 kB|45.94 ms|
|MobileNetV2 -> CS90 -> WQ32 -

In [74]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import tensorflow_model_optimization as tfmot
import tensorflow_datasets as tfds

%load_ext tensorboard

from os import path
import pathlib
import tempfile

# normalizing the images to [0, 1]
def normalize(image, label):
    return tf.cast(image, tf.float32) / 255., label

def random_crop(image):
    cropped_image = tf.image.random_crop(
        image, size=[256, 256, 3])

    return cropped_image

def random_jitter(image):
    # resizing to 286 x 286 x 3
    image = tf.image.resize(image, [286, 286],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)

    # randomly cropping to 256 x 256 x 3
    image = random_crop(image)

    # random mirroring
    image = tf.image.random_flip_left_right(image)

    return image

def preprocess_flowers_train(image, label):
    image = random_jitter(image)
    return image, label

# -------------------------------

def preprocess_flowers(image, label):
    image = tf.image.resize(image, [256, 256],
                          method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return image, label

def load_flowers_dataset():  
    ds_test, ds_info = tfds.load(name="tf_flowers", 
                                with_info=True,
                                split=['test'],
                                as_supervised=True)

#     ds_train = ds_train.map(normalize)    
#     ds_train = ds_train.map(preprocess_flowers)
#     ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
#     ds_validation = ds_validation.map(normalize)
#     ds_validation = ds_validation.map(preprocess_flowers)
    ds_train = None
    ds_validation= None
    ds_test = ds_test.map(normalize)
    ds_test = ds_test.map(preprocess_flowers)
    
    return ds_train, ds_validation, ds_test

def load_beans_datasets():
    (ds_train, ds_validation, ds_test), ds_info = tfds.load(
        'beans',
        split=['train', 'validation', 'test'],
        shuffle_files=True,
        as_supervised=True,
        with_info=True,
    )
    
    ds_train = ds_train.map(normalize)
    ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
    
    ds_validation = ds_validation.map(normalize)
    
    ds_test = ds_test.map(normalize)
    
    return ds_train, ds_validation, ds_test

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [75]:
new_model = tf.keras.models.load_model('flowers_models_optimized/EfficentNetB0_flowers_model_ConstantSparsity50.h5')

In [76]:
new_model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                      metrics=['accuracy'])

In [77]:
flowers_test = load_flowers_dataset()[2]
flowers_test = flowers_test.batch(1)
new_model.evaluate(flowers_test)

ValueError: Unknown split "test". Should be one of ['train'].